In [1]:
using LatticePhysics
using LatPhysPlottingPyPlot; using LatPhysReciprocalPlottingPyPlot;
using PyPlot; pygui(false);
using BenchmarkTools
using Traceur
using Random
using LinearAlgebra

In [2]:
function vector(
            bond    :: B,
            lattice :: LA
        ) :: Vector{Float64} where {N,L,B<:AbstractBond{L,N}, SU,BU,U<:AbstractUnitcell{SU,BU}, SL,LL,BL<:AbstractBond{LL,N},LA<:AbstractLattice{SL,BL,U}}

    # build the offset vector
    v = point(site(lattice,to(bond))) .- point(site(lattice,from(bond)))
    for i in 1:N
        v .+= wrap(bond)[i].*latticeVectors(lattice)[i]
    end
    # return the offset vector
    return v
end

function vector(
            bond     :: B,
            unitcell :: U
        ) :: Vector{Float64} where {N,L,B<:AbstractBond{L,N}, S,LU,BU<:AbstractBond{LU,N},U<:AbstractUnitcell{S,BU}}

    # build the offset vector
    v = point(site(unitcell,to(bond))) .- point(site(unitcell,from(bond)))
    for i in 1:N
        v .+= wrap(bond)[i].*latticeVectors(unitcell)[i]
    end
    # return the offset vector
    return v
end

vector (generic function with 2 methods)

In [3]:
import Random.shuffle!
function shuffle!(
            lattice :: L
        ) where {LB,N,LS,D,B<:AbstractBond{LB,N},S<:AbstractSite{LS,D},U,L<:AbstractLattice{S,B,U}}
    # make a new list of site indices
    indices_new = shuffle(collect(1:length(sites(lattice))))
    # make a new list of sites
    sites_new   = Vector{S}(undef, length(sites(lattice)))
    for s in 1:length(sites(lattice))
        sites_new[indices_new[s]] = site(lattice, s)
    end
    # reformat all bonds
    for b in bonds(lattice)
        from!(b, indices_new[from(b)])
        to!(b, indices_new[to(b)])
    end
    # set the sites
    sites!(lattice, sites_new)
    # shuffle all bonds
    bonds!(lattice, shuffle(bonds(lattice)))
end

shuffle! (generic function with 4 methods)

In [21]:
function rotateAroundZAxis!(lattice::AbstractLattice, angle::Real)
    # rotate the lattice vectors
    for lv in latticeVectors(lattice)
        # rotate and overwrite simulateously all components
        lv[1], lv[2] = cos(angle)*lv[1] + sin(angle)*lv[2]  ,   -sin(angle)*lv[1] + cos(angle)*lv[2]
    end
    # rotate the basis sites
    for p in point.(sites(lattice))
        # rotate and overwrite simulateously all components
        p[1], p[2] = cos(angle)*p[1] + sin(angle)*p[2]  ,   -sin(angle)*p[1] + cos(angle)*p[2]
    end
end

function rotateAroundZAxisDeg!(lattice::AbstractLattice, angle::Real)
    rotateAroundZAxis!(lattice, angle*pi/180.0)
end

function rotateAroundZAxis!(uc::AbstractUnitcell, angle::Real)
    # rotate the lattice vectors
    for lv in latticeVectors(uc)
        # rotate and overwrite simulateously all components
        lv[1], lv[2] = cos(angle)*lv[1] + sin(angle)*lv[2]  ,   -sin(angle)*lv[1] + cos(angle)*lv[2]
    end
    # rotate the basis sites
    for p in point.(sites(uc))
        # rotate and overwrite simulateously all components
        p[1], p[2] = cos(angle)*p[1] + sin(angle)*p[2]  ,   -sin(angle)*p[1] + cos(angle)*p[2]
    end
end

function rotateAroundZAxisDeg!(uc::AbstractUnitcell, angle::Real)
    rotateAroundZAxis!(uc, angle*pi/180.0)
end

rotateAroundZAxisDeg! (generic function with 2 methods)

In [157]:
m = 10
n = 2

uc = getUnitcellHoneycomb()

L1 = m*a1(uc) + n*a2(uc)
L2 = [cos(pi/3)*L1[1] + sin(pi/3)*L1[2]  ,   -sin(pi/3)*L1[1] + cos(pi/3)*L1[2]]

L1_perp = [-L1[2], L1[1]]
L2_perp = [-L2[2], L2[1]]

# functions to find out the prefactors of new lattice vectors
alpha_L1(p) = dot(p,L2_perp)/dot(L1,L2_perp)
alpha_L2(p) = dot(p,L1_perp)/dot(L2,L1_perp)
# function that determines the new moiree cell
moiree_uc_shape(p) = (0 <= alpha_L1(p) < 1) && (0 <= alpha_L2(p) < 1)

# create first layer
uc1 = deepcopy(uc)
label!.(sites(uc1), 1)
lt1 = getLatticeInShape(uc1, moiree_uc_shape)

# create second layer
uc2 = deepcopy(uc)
label!.(sites(uc2), 2)
rotateAroundZAxis!(uc2, acos(0.5*(m*m + n*n + 4*m*n)/(m*m + n*n + m*n)))
lt2 = getLatticeInShape(uc2, moiree_uc_shape)

# create a shared lattice with sites and bonds of both layers
lt_shared = deepcopy(lt1)
for b in bonds(lt2)
    from!(b, from(b)+length(sites(lt1)))
    to!(b, to(b)+length(sites(lt1)))
end
append!(sites(lt_shared), sites(lt2))
append!(bonds(lt_shared), bonds(lt2))

# make a new unitcell out of this
uc_moiree = deepcopy(uc)
latticeVectors!(uc_moiree, [L1,L2])
sites!(uc_moiree, sites(lt_shared))

# maximum bond length (NN: 0.567...)
max_length = 5
# height different between layers
a_0 = 0.5

# fill the bonds of the moiree unitcell
bonds_moiree = Bond{typeof(label(bonds(uc)[1])), 2}[
    newBond(
        Bond{typeof(label(bonds(uc)[1])), 2},
        i,j,
        getDefaultLabelN(typeof(label(bonds(uc)[1])), label(site(uc_moiree,i))*label(site(uc_moiree,j))),
        (dn1,dn2)
    )
    for i in 1:length(sites(uc_moiree))
    for j in 1:length(sites(uc_moiree))
    for dn1 in [-1,0,1]
    for dn2 in [-1,0,1]
    if (norm(point(site(uc_moiree,j)) .+ dn1*L1 .+ dn2*L2 .- point(site(uc_moiree,i)))^2 + (label(site(uc_moiree,i))==label(site(uc_moiree,j)) ? 0 : 1)*a_0^2) < max_length^2
]
bonds!(uc_moiree, bonds_moiree);

# show the moiree unitcell
uc_moiree

Unitcell object
--> type Unitcell{Site{Int64,2},Bond{Int64,2}}
--> 610 sites of type Site{Int64,2}
--> 269451 bonds of type Bond{Int64,2}

In [156]:
moiree_lattice = getLatticeOpen(uc_moiree, 3)
pygui(true)
plotLattice(moiree_lattice, colorcode_bonds=Dict(1=>[0,0,0], 4=>[180,180,180], 2=>[255,0,0]));
moiree_lattice

Lattice object
--> type Lattice{Site{Int64,2},Bond{Int64,0},Unitcell{Site{Int64,2},Bond{Int64,2}}}
--> 5490 sites of type Site{Int64,2}
--> 45234 bonds of type Bond{Int64,0}
--> unitcell of type Unitcell{Site{Int64,2},Bond{Int64,2}}

In [ ]:
mutable struct TBGHamiltonian
    
end